## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import thr API key.
import sys
sys.path.append('..')
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv(os.path.join("..", "Weather_Database", "WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,62.80,90,5,10.96,clear sky
1,1,Bluff,NZ,-46.6000,168.3333,60.19,63,7,18.95,clear sky
2,2,Cabedelo,BR,-6.9811,-34.8339,79.47,78,56,10.87,broken clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,39.07,74,100,3.67,overcast clouds
4,4,Mkushi,ZM,-13.6202,29.3939,61.61,95,99,6.80,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Cabedelo,BR,-6.9811,-34.8339,79.47,78,56,10.87,broken clouds
5,5,Bathsheba,BB,13.2167,-59.5167,79.41,73,40,19.57,scattered clouds
9,9,Alugan,PH,12.2188,125.4808,79.75,89,100,10.33,overcast clouds
10,10,Phan Thiet,VN,10.9333,108.1000,77.76,78,86,2.13,overcast clouds
14,14,Hilo,US,19.7297,-155.0900,78.53,81,100,6.91,overcast clouds
16,16,Arraial Do Cabo,BR,-22.9661,-42.0278,75.81,82,26,4.92,scattered clouds
17,17,Rikitea,PF,-23.1203,-134.9692,79.29,73,3,16.87,clear sky
18,18,Auki,SB,-8.7676,160.7034,79.14,85,87,2.91,light rain
21,21,Georgetown,MY,5.4112,100.3354,80.53,83,20,0.00,thunderstorm
23,23,Kawardha,IN,22.0167,81.2500,82.60,27,35,3.60,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Cabedelo,BR,-6.9811,-34.8339,79.47,78,56,10.87,broken clouds
5,5,Bathsheba,BB,13.2167,-59.5167,79.41,73,40,19.57,scattered clouds
9,9,Alugan,PH,12.2188,125.4808,79.75,89,100,10.33,overcast clouds
10,10,Phan Thiet,VN,10.9333,108.1000,77.76,78,86,2.13,overcast clouds
14,14,Hilo,US,19.7297,-155.0900,78.53,81,100,6.91,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Cabedelo,BR,79.47,broken clouds,-6.9811,-34.8339,
5,Bathsheba,BB,79.41,scattered clouds,13.2167,-59.5167,
9,Alugan,PH,79.75,overcast clouds,12.2188,125.4808,
10,Phan Thiet,VN,77.76,overcast clouds,10.9333,108.1000,
14,Hilo,US,78.53,overcast clouds,19.7297,-155.0900,
16,Arraial Do Cabo,BR,75.81,scattered clouds,-22.9661,-42.0278,
17,Rikitea,PF,79.29,clear sky,-23.1203,-134.9692,
18,Auki,SB,79.14,light rain,-8.7676,160.7034,
21,Georgetown,MY,80.53,thunderstorm,5.4112,100.3354,
23,Kawardha,IN,82.60,scattered clouds,22.0167,81.2500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat =row["Lat"]
    lng =row["Lng"]    
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except(IndexError):

        print("Hotel not found... skipping.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} F</dd>
</d1>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))